# K-Means Anchors Ratios Tutorial

In [1]:
import json
import numpy as np
import os

%load_ext autoreload
%autoreload 2

## Get COCO train 2017 instances

In [2]:
INSTANCES_PATH = "./annotations/instances_train2017.json"

!pip install -r requirements.txt
if not os.path.exists(INSTANCES_PATH):
    !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
    !unzip annotations_trainval2017.zip

In [3]:
with open(INSTANCES_PATH) as f:
    instances = json.load(f)

## Get optimal anchors ratios

In [4]:
## change the following parameters according to your model:

# efficientdet0 input size
INPUT_SIZE = 512
# efficientdet default sizes
ANCHORS_SIZES = (np.sqrt([2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]) * 
                 np.array([32, 64, 128, 256, 512])[:, np.newaxis]).flatten().tolist()
ANCHORS_SIZES

[32.0,
 35.91878554589994,
 40.31747359663594,
 64.0,
 71.83757109179987,
 80.63494719327188,
 128.0,
 143.67514218359975,
 161.26989438654377,
 256.0,
 287.3502843671995,
 322.53978877308754,
 512.0,
 574.700568734399,
 645.0795775461751]

In [5]:
from kmeans_anchors_ratios import get_optimal_anchors_ratios

anchors_ratios = get_optimal_anchors_ratios(
    instances,
    anchors_sizes=ANCHORS_SIZES,
    input_size=INPUT_SIZE,
    normalizes_bboxes=True,
    num_runs=3,
    num_anchors_ratios=3,
    max_iter=300,
    iou_threshold=0.5,
    min_size=0,
    decimals=1,
    default_anchors_ratios=[(0.7, 1.4), (1.0, 1.0), (1.4, 0.7)]
)

[06/13 12:51:09] Starting the calculation of the optimal anchors ratios
[06/13 12:51:09] Extracting and preprocessing bounding boxes
[06/13 12:51:12] Discarding 2 bounding boxes with size lower or equal to 0
[06/13 12:51:13] K-Means (3 runs): 100%|██████████████████| 3/3 [00:34<00:00, 11.43s/it]	Runs avg. IoU: 80.48% ± 0.00% (mean ± std. dev. of 3 runs, 0 skipped)
	Avg. IoU between bboxes and their most similar anchors after norm. them to make their area equal (only ratios matter): 80.48%

[06/13 12:51:51] Default anchors ratios: [(0.7, 1.4), (1.0, 1.0), (1.4, 0.7)]
	Avg. IoU between bboxes and their most similar default anchors, no norm. (both ratios and sizes matter): 60.57%
	Num. bboxes without similar default anchors (IoU >= 0.5):  251107/860001 (29.20%)
[06/13 12:51:58] K-Means anchors ratios: [(0.6, 1.5), (1.0, 1.0), (1.4, 0.7)]
	Avg. IoU between bboxes and their most similar K-Means anchors, no norm. (both ratios and sizes matter): 61.02%
	Num. bboxes without similar K-Means anc

## Generate anchors given ratios and sizes

In [6]:
from kmeans_anchors_ratios import generate_anchors_given_ratios_and_sizes


anchors = generate_anchors_given_ratios_and_sizes(anchors_ratios, ANCHORS_SIZES)
print("Anchors:")
print(anchors)

Anchors:
[[ 19.2         48.        ]
 [ 32.          32.        ]
 [ 44.8         22.4       ]
 [ 21.55127133  53.87817832]
 [ 35.91878555  35.91878555]
 [ 50.28629976  25.14314988]
 [ 24.19048416  60.47621039]
 [ 40.3174736   40.3174736 ]
 [ 56.44446304  28.22223152]
 [ 38.4         96.        ]
 [ 64.          64.        ]
 [ 89.6         44.8       ]
 [ 43.10254266 107.75635664]
 [ 71.83757109  71.83757109]
 [100.57259953  50.28629976]
 [ 48.38096832 120.95242079]
 [ 80.63494719  80.63494719]
 [112.88892607  56.44446304]
 [ 76.8        192.        ]
 [128.         128.        ]
 [179.2         89.6       ]
 [ 86.20508531 215.51271328]
 [143.67514218 143.67514218]
 [201.14519906 100.57259953]
 [ 96.76193663 241.90484158]
 [161.26989439 161.26989439]
 [225.77785214 112.88892607]
 [153.6        384.        ]
 [256.         256.        ]
 [358.4        179.2       ]
 [172.41017062 431.02542655]
 [287.35028437 287.35028437]
 [402.29039811 201.14519906]
 [193.52387326 483.80968316]
 [322

## Get bounding boxes adapted to the input size

In [7]:
from kmeans_anchors_ratios import get_bboxes_adapted_to_input_size


resized_bboxes = get_bboxes_adapted_to_input_size(instances, input_size=INPUT_SIZE)
resized_bboxes = resized_bboxes[resized_bboxes.prod(axis=1) > 0]  # remove 0 size
print("Bounding boxes adapted to the input size (first 5):")
print(resized_bboxes[:5])

Bounding boxes adapted to the input size (first 5):
[[ 62.168  56.704]
 [119.512  30.84 ]
 [128.     46.248]
 [100.752  18.168]
 [ 53.176  53.4  ]]


## Get the avg. IoU between the bounding boxes and their closest anchors

In [8]:
from kmeans_anchors_ratios import average_iou


avg_iou = average_iou(resized_bboxes, anchors)
print(f"Avg. IoU: {100 * avg_iou:.2f}%")

Avg. IoU: 61.02%


## Get annotations whose bounding boxes don't have similar anchors

In [9]:
from kmeans_anchors_ratios import get_annotations_without_similar_anchors


annotations = get_annotations_without_similar_anchors(
    instances,
    anchors_ratios,
    anchors_sizes=ANCHORS_SIZES,
    input_size=INPUT_SIZE,
    iou_threshold=0.5,
    min_size=0,
)

bboxes = [ann["bbox"][-2:] for ann in annotations]  # widths and heights
print("Bounding boxes without similar anchors (first 5):")
print(bboxes[:5])

instances_without_similar_anchors = instances.copy()
instances_without_similar_anchors["annotations"] = annotations
resized_bboxes = get_bboxes_adapted_to_input_size(instances_without_similar_anchors, input_size=INPUT_SIZE)
print("Bounding boxes without similar anchors adapted to the input size (first 5):")
print(resized_bboxes[:5])

Bounding boxes without similar anchors (first 5):
[[125.94, 22.71], [26.17, 19.49], [13.28, 9.36], [34.03, 22.54], [29.63, 25.24]]
Bounding boxes without similar anchors adapted to the input size (first 5):
[[100.752  18.168]
 [ 20.936  15.592]
 [ 10.624   7.488]
 [ 27.224  18.032]
 [ 23.704  20.192]]
